In [1]:
# Analysis of Retirement with Mutual Funds
# Analysis period from 1999 Jan to 2020 May

#Importing libraries
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

In [2]:
# install xlrd to read spreadsheets
!pip install xlrd
import xlrd
print('xlrd installed!')

xlrd installed!


In [3]:
# reading excel file into data frame
df_data = pd.read_excel('nav_history.xlsx',
                       sheet_name='Sheet1')

print ('Data read into a pandas dataframe!')

Data read into a pandas dataframe!


In [4]:
# checking data frame
df_data.head()

,Date,Liquid Fund – NAV,Equity Fund – NAV
0,"Jan 01, 1999",11.41,9.29
1,"Feb 01, 1999",11.52,10.23
2,"Mar 01, 1999",11.62,11.64
3,"Apr 05, 1999",11.74,12.62
4,"May 03, 1999",11.84,11.14


In [5]:
# view df details
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               257 non-null    object 
 1   Liquid Fund – NAV  257 non-null    float64
 2   Equity Fund – NAV  257 non-null    float64
dtypes: float64(2), object(1)
memory usage: 6.1+ KB


In [6]:
# Initial monthly income at 1999 Jan 1 is set to ₹12,000. Reason: at 7% inflation, ₹12,000 ~ ₹50,000 as on 2020
# Initial investment is set so that total withdrawal per year is 4% of capital. in this case ₹36 Lakhs
# Asset allocation is set to 60% Equity and 40% Debt

init_inv=3600000
init_eq_per=0.6
init_eq=init_inv*init_eq_per
init_dt=init_inv-init_eq

monthly_income=[] # creating a list for storing monthy income for every year
year_num=0 # to track year in progress
monthly_income.append(12000)

In [10]:

# adding 6 new empty columns
df_data['Liquid fund – Units']=""
df_data['Equity fund – Units']=""
df_data['Redemption']=""
df_data['Liquid fund – Value']=""
df_data['Equity fund – Value']=""
df_data['Equity percent']=""

#initializing first row
df_data.loc[0,'Liquid fund – Units']=(init_dt-monthly_income[year_num])/df_data.loc[0,'Liquid Fund – NAV']
df_data.loc[0,'Equity fund – Units']=init_eq/df_data.loc[0,'Equity Fund – NAV']
df_data.loc[0,'Redemption']=monthly_income[year_num]
df_data.loc[0,'Liquid fund – Value']=df_data.loc[0,'Liquid fund – Units']*df_data.loc[0,'Liquid Fund – NAV']
df_data.loc[0,'Equity fund – Value']=df_data.loc[0,'Equity fund – Units']*df_data.loc[0,'Equity Fund – NAV']
df_data.loc[0,'Equity percent']=(100*df_data.loc[0,'Equity fund – Value'])/(df_data.loc[0,'Equity fund – Value']+df_data.loc[0,'Liquid fund – Value'])

# number of rows
var_rows=len(df_data.index)

for i in range(1,var_rows):
    if(i%12==11): # when 1 year is completed
        #code
        print('test')
    else:
        #code
        df_data.loc[i,'Liquid fund – Units']=df_data.loc[i-1,'Liquid fund – Units']-(monthly_income[year_num]/df_data.loc[i,'Liquid Fund – NAV'])
        df_data.loc[i,'Equity fund – Units']=df_data.loc[i-1,'Equity fund – Units']
        df_data.loc[i,'Redemption']=monthly_income[year_num]
        df_data.loc[i,'Liquid fund – Value']=df_data.loc[i,'Liquid fund – Units']*df_data.loc[i,'Liquid Fund – NAV']
        df_data.loc[i,'Equity fund – Value']=df_data.loc[i,'Equity fund – Units']*df_data.loc[i,'Equity Fund – NAV']
        df_data.loc[i,'Equity percent']=(100*df_data.loc[i,'Equity fund – Value'])/(df_data.loc[i,'Equity fund – Value']+df_data.loc[0,'Liquid fund – Value'])
    #end else
#end for

test


UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')

In [17]:
df_data.head()

,Date,Liquid Fund – NAV,Equity Fund – NAV,Liquid fund – Units,Equity fund – Units,Redemption,Liquid fund – Value,Equity fund – Value,Equity percent
0,"Jan 01, 1999",11.41,9.29,125153,232508,12000,1.428e+06,2.16e+06,60.2007
1,"Feb 01, 1999",11.52,10.23,124112,232508,12000,1.42977e+06,2.37856e+06,62.4858
2,"Mar 01, 1999",11.62,11.64,123079,232508,12000,1.43018e+06,2.70639e+06,65.4605
3,"Apr 05, 1999",11.74,12.62,122057,232508,12000,1.43295e+06,2.93425e+06,67.2646
4,"May 03, 1999",11.84,11.14,121043,232508,12000,1.43315e+06,2.59014e+06,64.4612
